DS 8014 Graph Mining
Assignment 5

Gorkem Sarp;
Richa Sharma

Que 2:

Some of the code was taken from our labs

In [1]:
#! pip install python-igraph
#! pip install plfit
#! pip install partition-igraph
#! pip3 install -U pip
#! pip3 install -U setuptools
#! pip3 install --no-cache-dir cairocffi
#! pip install umap-learn

In [1]:
import igraph as ig
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LinearRegression
from collections import Counter
import os
import umap
import pickle
import partition_igraph
import subprocess
from sklearn.metrics import adjusted_mutual_info_score as AMI

## we used those for the book, but you can change to other colors
cls_edges = 'gainsboro'
cls = ['silver','dimgray','black']

In [2]:
## read the GitHub edge list as tuples and build undirected graph
D = pd.read_csv('musae_git_edges.csv')
tuples = [tuple(x) for x in D.values]
z = ig.Graph.TupleList(tuples, directed = False)

## read node features
X = pd.read_csv('musae_git_target.csv')

## map node names in edgelist to indices in the graph
idx = [int(i) for i in z.vs['name']]
sorterIndex = dict(zip(idx,range(len(idx))))
X['Rank'] = X['id'].map(sorterIndex) 
X.sort_values(['Rank'], ascending=[True],inplace=True)
X.dropna(inplace=True)

cls = ['grey','black'] ## node colors
lbl = ['web','ml']     ## node labels
z.es['color'] = 'grey' ## edge color

## there are 2 node types: ml or web
z.vs['color'] = [cls[i] for i in list(X['ml_target'])]
z.vs['lbl'] = [lbl[i] for i in list(X['ml_target'])]

## community labels 0 (web) or 1 (ml)
z.vs['comm'] = [i for i in list(X['ml_target'])]
#print(z.vs['comm'])

## ECG

In [7]:
ans = AMI((z.community_ecg().membership), (z.community_multilevel().membership))
ans2 = AMI((z.community_ecg().membership), (z.community_infomap().membership))
ans3 = AMI((z.community_ecg().membership),(z.community_label_propagation().membership))
print("AMI between ECG Algorithm and Louvain:", ans)
print("AMI between ECG Algorithm and infomap:", ans2)
print("AMI between ECG Algorithm and label propagation:", ans3)

AMI between ECG Algorithm and Louvain: 0.5253200932781608
AMI between ECG Algorithm and infomap: 0.43490038269117737
AMI between ECG Algorithm and label propagation: 0.16601180663226106


In [9]:
print("ECG Algorithm seem most similar to Louvain among the tested algorithms")

ECG Algorithm seem most similar to Louvain among the tested algorithms


## Louvain

In [10]:
ans=AMI((z.community_multilevel().membership),(z.community_infomap().membership))
ans2=AMI((z.community_multilevel().membership),(z.community_label_propagation().membership))
print("AMI between Louvain and infomap :", ans)
print("AMI between Louvain and label propagation: ", ans2)

AMI between Louvain and infomap : 0.32719972860524676
AMI between Louvain and label propagation:  0.005052793563481771


In [11]:
print("From the above comparisons, and the previous one, Louvain is also most similar to ECG")

From the above comparisons, and the previous one, Louvain is also most similar to ECG


## Infomap

In [4]:
ans= AMI((z.community_infomap().membership),(z.community_label_propagation().membership))
print("AMI between infomap Algorithm and label propagation:", ans)

AMI between infomap Algorithm and label propagation: 0.08494701976070698


In [12]:
print("From the above comparisons, and the previous ones, infomap is also closest with ECG")

From the above comparisons, and the previous ones, infomap is also closest with ECG


## Label propagation

In [16]:
print("From the previous comparisons, label propogation is also closest with ECG")

From the previous comparisons, label propogation is also closest with ECG


## Girvan Newman 

In [ ]:
## compute modularity at each possible cut and plot for Girvan newman
gn = z.community_edge_betweenness()
q = []

#limited number of n-1 cluster search (with n being number of nodes) to 10 nodes, given this 
#function takes too long to run
for i in np.arange(10): 
    q.append(z.modularity(gn.as_clustering(n=i+1)))
plt.plot(np.arange(1,1+z.vcount()),q,'o-',color='black')
plt.xlabel('number of clusters',fontsize=14)
plt.ylabel('modularity',fontsize=14);

In [ ]:
## Assuming n=2 clusters, similar to ground truth
GN1 =  AMI((z.community_edge_betweenness().as_clustering(n=2).membership), (z.community_ecg().membership)
GN2 =  AMI((z.community_edge_betweenness().as_clustering(n=2).membership), (z.community_multilevel().membership)        
GN3 =  AMI((z.community_edge_betweenness().as_clustering(n=2).membership), (z.community_infomap().membership)
GN4 =  AMI((z.community_edge_betweenness().as_clustering(n=2).membership), (z.community_label_propagation().membership)         
GN5 =  AMI((z.community_edge_betweenness().as_clustering(n=2).membership), (z.community_fastgreedy().as_clustering().membership)         
          
print("AMI of Girvan Newman Algorithm and ECG:", GN1))
print("AMI of Girvan Newman Algorithm and Louvain", GN2)
print("AMI of Girvan Newman Algorithm and Infomap", GN3)
print("AMI of Girvan Newman Algorithm and Label propagation", GN4)
print("AMI of Girvan Newman Algorithm and CNM", GN5)

##  CNM

In [ ]:
CNM1 =  AMI((z.community_fastgreedy().as_clustering().membership), (z.community_ecg().membership)
CNM2 =  AMI((z.community_fastgreedy().as_clustering().membership), (z.community_multilevel().membership)        
CNM3 =  AMI((z.community_fastgreedy().as_clustering().membership), (z.community_infomap().membership)
CNM4 =  AMI((z.community_fastgreedy().as_clustering().membership), (z.community_label_propagation().membership)         
CNM5 =  AMI((z.community_fastgreedy().as_clustering().membership), (z.community_edge_betweenness().as_clustering(n=2).membership))
            
print("AMI of CNM and ECG:", CNM1))
print("AMI of CNM and Louvain", CNM2)
print("AMI of CNM and Infomap", CNM3)
print("AMI of CNM and Label propagation", CNM4)
print("AMI of CNM and Girvan Newman Algorithm", CNM5)           
    

In [15]:
print("We couldn't compute the AMI of Girvan Newman and CNM since the functions kept running")

We couldn't compute the AMI of Girvan Newman and CNM since the functions kept running
